# Task definition
Implement LSTM Sentiment Tagger for imdb reviews dataset.

1. (5pt) Fill missing code below
    * 1pt implement vectorization
    * 2pt implement \_\_init\_\_ and forward methods of models
    * 2pt implement collate function
2. (4pt) Implement training loop, choose proper loss function, use clear ml for max points.
    * 2pts is a baseline for well written, working code
    * 2pts if clear ml used properly
3. (3pt) Train the models (find proper hyperparams). Make sure you are not overfitting or underfitting. Visualize training of your best model (plot training, and test loss/accuracy in time). Your model should reach at least 87% accuracy. For max points it should exceed 89%. 
    * 1pt for accuracy above 89%
    * 1pt for accuracy above 87%
    * 1pt for visualizations

Remarks:
* Use embeddings of size 50
* Use 0.5 threshold when computing accuracy.
* Use supplied dataset for training and evaluation.
* You do not have to use validation set.
* You should monitor overfitting during training.
* For max points use clear ml to store and manage logs from your experiments. 
* We encourage to use pytorch lightning library (Addtional point for using it - however the sum must not exceed 12)

[Clear ML documentation](https://clear.ml/docs/latest/docs/)

[Clear ML notebook exercise from bootcamp](https://colab.research.google.com/drive/1wtLb4gg8beLS7smcyJlOZppn6_rQvSxL?usp=sharing)

In [4]:
!pip install clearml plotly-express nbformat kaleido

import os
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torchtext
from clearml import Task, Model

import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import Trainer

import pickle

import plotly.express as px
import nbformat

In [5]:

web_server = 'https://app.community.clear.ml'
api_server = 'https://api.community.clear.ml'
files_server = 'https://files.community.clear.ml'
access_key = ''#@param {type:"string"}
secret_key = ''#@param {type:"string"}

Task.set_credentials(web_host=web_server,
                     api_host=api_server,
                     files_host=files_server,
                     key=access_key,
                     secret=secret_key)

In [6]:
# !pip install gdown
# !gdown https://drive.google.com/uc?id=1hK-3iiRPlbePb99Fe-34LJNZ5yB-nduq
# !tar -xvzf imdb_dataset.gz
data = pd.read_csv("imdb_dataset.csv")

In [7]:
PADDING_VALUE = 0
PADDING_OBJECT = object()

def flatten(list_of_lists):
  result = []
  for l in list_of_lists:
    result.extend(l)
  return result

class NaiveVectorizer:
    def __init__(self, tokenized_data, **kwargs):
        """Converts data from string to vector of ints that represent words. 
        Prepare lookup dict (self.wv) that maps token to int. Reserve index 0 for padding.
        """
        tokenized_data = [seq.split() for seq in tokenized_data]
        self.wv = {}
        self.wv[PADDING_OBJECT] = PADDING_VALUE
        ### Your code goes here ###
        for i, token in enumerate(set(flatten(tokenized_data)), start = PADDING_VALUE + 1):
          self.wv[token] = i  
        
        self.vocab_size = len(self.wv)
        ##################################

    def vectorize(self, tokenized_seq):
        """Converts sequence of tokens into sequence of indices.
        If the token does not appear in the vocabulary(self.wv) it is ommited
        Returns torch tensor of shape (seq_len,) and type long."""
        ### Your code goes here ###
        indices_list = [self.wv[token] for token in tokenized_seq if token in self.wv]
        return torch.tensor(indices_list, dtype=torch.long)

        ##################################

class ImdbDataset(Dataset):
    SPLIT_TYPES = ["train", "test", "unsup"]

    def __init__(self, data, preprocess_fn, split="train"):
        super(ImdbDataset, self).__init__()
        if split not in self.SPLIT_TYPES:
            raise AttributeError(f"No such split type: {split}")

        self.split = split
        self.label = [i for i, c in enumerate(data.columns) if c == "sentiment"][0]
        self.data_col = [i for i, c in enumerate(data.columns) if c == "tokenized"][0]
        self.data = data[data["split"] == self.split]
        self.preprocess_fn = preprocess_fn

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.preprocess_fn(self.data.iloc[idx, self.data_col].split())
        label = self.data.iloc[idx, self.label]
        return (seq, label)

# naive_vectorizer = NaiveVectorizer(data.loc[data["split"] == "train", "tokenized"])
with open('vectorizer.p', 'rb') as f:
    naive_vectorizer = pickle.load(f)

def get_datasets():
    train_dataset = ImdbDataset(data, naive_vectorizer.vectorize)
    test_dataset = ImdbDataset(data, naive_vectorizer.vectorize, split="test")

    return train_dataset, test_dataset


def custom_collate_fn(pairs):
    """This function is supposed to be used by dataloader to prepare batches
    Input: list of tuples (sequence, label)
    Output: sequences_padded_to_the_same_lenths, original_lenghts_of_sequences, lables.
    torch.nn.utils.rnn.pad_sequence might be usefull here
    """
    ### Your code goes here ###
    sequences_python = [seq for seq, _ in pairs]
    labels = [label for _, label in pairs]
    lengths = [len(seq) for seq in sequences_python]
    sequences_torch = torch.nn.utils.rnn.pad_sequence(sequences_python).squeeze().T

    #################################
    return sequences_torch, torch.tensor(lengths, dtype=int), torch.tensor(labels).reshape([-1, 1])

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_number_of_correct_predictions(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()
    return acc

train_set, test_set = get_datasets()

In [9]:
"""Implement LSTMSentimentTagger. 
The model should use a LSTM module.
Use torch.nn.utils.rnn.pack_padded_sequence to optimize processing of sequences.
When computing vocab_size of embedding layer remeber that padding_symbol counts to the vocab.
Use sigmoid activation function.
"""
class LSTMSentimentTagger(pl.LightningModule):
    def __init__(self, vocab_size, classes, embedding_dim=50, hidden_dim=2, layers_count=5, lr=1e-3, batch_size=64):
        super().__init__()
        ### Your code goes here ###
        self.save_hyperparameters()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.classes = classes
        self.layers_count = layers_count
        self.lr = lr
        self.batch_size = batch_size
        self.validation_correct = 0
        self.validation_total = 0

        self._init_modules()
        #################################
    
    def _init_modules(self):
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(input_size = self.embedding_dim, hidden_size=self.hidden_dim, num_layers=self.layers_count, batch_first=True)
        self.fc = nn.Linear(in_features = self.hidden_dim, out_features = self.classes)

    def forward(self, sentences, lengths):
        embeddings = torch.nn.utils.rnn.pack_padded_sequence(self.embedding(sentences), lengths.to('cpu'), batch_first=True, enforce_sorted=False)
        thru_lstm, (h_n, c_n) = self.lstm(embeddings)
        thru_lstm, _ = torch.nn.utils.rnn.pad_packed_sequence(thru_lstm, batch_first=True)
        number = self.fc(thru_lstm[torch.arange(thru_lstm.size(0)), lengths-1].squeeze())
        #################################
        return number
    
    def training_step(self, batch, batch_idx):
        seqs, lengths, labels = batch
        logits = self(seqs, lengths)
        loss = F.binary_cross_entropy_with_logits(logits, labels)
        acc = get_number_of_correct_predictions(logits, labels)
        self.log('train_loss', loss, on_epoch=True, on_step=True)
        self.log('train_correct', acc, on_epoch=True, on_step=True)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        seqs, lengths, labels = batch
        logits = self(seqs, lengths)
        loss = F.binary_cross_entropy_with_logits(logits, labels)
        acc = get_number_of_correct_predictions(logits, labels)
        self.log('val_loss', loss, on_epoch=True, on_step=True)
        self.log('val_correct', acc, on_epoch=True, on_step=True)
        return {'loss': loss}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    
    def train_dataloader(self):
        # num_workers = 8 because my CPU has 8 threads
        return DataLoader(train_set, self.batch_size, collate_fn=custom_collate_fn, shuffle=True, num_workers=8)

    def val_dataloader(self):
        return DataLoader(test_set, self.batch_size, collate_fn=custom_collate_fn, num_workers=8)


# Trainig loop and visualizations


In [7]:
config = {
    'lr': .0001,
    'hidden_dim': 200,
    'layers_count': 6,
    'embedding_dim': 50,
    'batch_size': 64,
    'n_epochs': 15,
}

for HIDDEN_DIM in [200, 100, 75]:
    for LAYERS_COUNT in [6, 5, 4]:
        for LR in [.0001, .0003, .00003]:
            config['hidden_dim'] = HIDDEN_DIM
            config['layers_count'] = LAYERS_COUNT
            config['lr'] = LR

            task = Task.init(project_name='lightning2', task_name='lightning example')
            task.connect(config)

            trainer = Trainer(gpus=1 if torch.cuda.is_available() else 0, max_epochs=config['n_epochs'])
            model = LSTMSentimentTagger(vocab_size=naive_vectorizer.vocab_size, classes=1, 
                                        embedding_dim=config['embedding_dim'], 
                                        hidden_dim=config['hidden_dim'], 
                                        layers_count=config['layers_count'],
                                        lr=config['lr'], 
                                        batch_size=config['batch_size'])
            trainer.fit(model)
            task.mark_completed()
            task.close()



ClearML Task: overwriting (reusing) task id=17ebdc2df698445da7995cd5bd3723b9
2022-01-09 02:05:45,009 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.community.clear.ml/projects/b62c3994002846a68db49133adfa829f/experiments/17ebdc2df698445da7995cd5bd3723b9/output/log


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 4.4 M 
1 | lstm      | LSTM      | 1.8 M 
2 | fc        | Linear    | 201   
----------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.645    Total estimated model params size (MB)


Epoch 14: 100%|██████████| 782/782 [02:47<00:00,  4.68it/s, loss=0.246, v_num=68]
ClearML Task: created new task id=a57664e9bfd4448aa652dfc581294632
ClearML results page: https://app.community.clear.ml/projects/b62c3994002846a68db49133adfa829f/experiments/a57664e9bfd4448aa652dfc581294632/output/log
2022-01-09 02:48:24,162 - clearml.Task - INFO - No repository found, storing script code instead


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 4.4 M 
1 | lstm      | LSTM      | 1.8 M 
2 | fc        | Linear    | 201   
----------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.645    Total estimated model params size (MB)


Epoch 14: 100%|██████████| 782/782 [02:47<00:00,  4.67it/s, loss=0.692, v_num=69]
ClearML Task: created new task id=15fa246119a84950bf3f6c2b5a0b8911
ClearML results page: https://app.community.clear.ml/projects/b62c3994002846a68db49133adfa829f/experiments/15fa246119a84950bf3f6c2b5a0b8911/output/log
2022-01-09 03:30:33,610 - clearml.Task - INFO - No repository found, storing script code instead


...

Epoch 14: 100%|██████████| 782/782 [01:17<00:00, 10.12it/s, loss=0.247, v_num=92]
ClearML Task: created new task id=b9c679bc2a354348994ce77710475ec4
ClearML results page: https://app.community.clear.ml/projects/b62c3994002846a68db49133adfa829f/experiments/b9c679bc2a354348994ce77710475ec4/output/log
2022-01-09 14:09:46,861 - clearml.Task - INFO - No repository found, storing script code instead


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 4.4 M 
1 | lstm      | LSTM      | 174 K 
2 | fc        | Linear    | 76    
----------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
18.106    Total estimated model params size (MB)


Epoch 14: 100%|██████████| 782/782 [01:17<00:00, 10.08it/s, loss=0.117, v_num=93] 
ClearML Task: created new task id=1425eeb729474e2bb2dabd62eb5b6f22
ClearML results page: https://app.community.clear.ml/projects/b62c3994002846a68db49133adfa829f/experiments/1425eeb729474e2bb2dabd62eb5b6f22/output/log
2022-01-09 14:29:22,317 - clearml.Task - INFO - No repository found, storing script code instead


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 4.4 M 
1 | lstm      | LSTM      | 174 K 
2 | fc        | Linear    | 76    
----------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
18.106    Total estimated model params size (MB)


Epoch 14: 100%|██████████| 782/782 [01:18<00:00,  9.98it/s, loss=0.412, v_num=94]
2022-01-09 14:49:02,719 - clearml.Task - INFO - Waiting to finish uploads
2022-01-09 14:49:03,440 - clearml.Task - INFO - Finished uploading


In [ ]:
# Save the vectorizer

# import pickle

# with open('vectorizer.p', 'wb') as f:
#     pickle.dump(naive_vectorizer, f)

In [41]:
# Best model (on val/test set) out of all training sessions

task = Task.get_task('4dc00e12927d4ea7b66d4954f345f404')
clearml_model = Model('79b4c50e2cd444b79b18e25a56efc6bb')
# model = LSTMSentimentTagger.load_from_checkpoint(clearml_model.url)
model = LSTMSentimentTagger.load_from_checkpoint('./model.ckpt')

In [11]:
model

LSTMSentimentTagger(
  (embedding): Embedding(87028, 50)
  (lstm): LSTM(50, 100, num_layers=4, batch_first=True)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [12]:
trainer = Trainer(gpus=1 if torch.cuda.is_available() else 0)

test_data_loader = DataLoader(test_set, model.batch_size, collate_fn=custom_collate_fn, num_workers=8)

model.eval()
correct_predictions = 0
all_predictions = 0
for batch in test_data_loader:
    seqs, lengths, labels = batch
    logits = model(seqs, lengths)
    correct_predictions += get_number_of_correct_predictions(logits, labels)
    all_predictions += labels.shape[0]
    print('.', end='')
print()
print(f"Model accuracy on test set: {(correct_predictions / all_predictions) * 100}%")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


.......................................................................................................................................................................................................................................................................................................................................................................................................
Model accuracy on test set: 87.37999725341797%


In [36]:
scalars = task.get_reported_scalars()

def save_plot(scalars, key):
    actual_dict = scalars[key][key]
    x = actual_dict['x']
    y = actual_dict['y']
    fig = px.line(x=x, y=y, title=key)
    fig.write_image(f'./images/{key}.png')

save_plot(scalars, 'val_correct_epoch')
save_plot(scalars, 'val_correct_step')
save_plot(scalars, 'val_loss_epoch')
save_plot(scalars, 'val_loss_step')

save_plot(scalars, 'train_correct_epoch')
save_plot(scalars, 'train_correct_step')
save_plot(scalars, 'train_loss_epoch')
save_plot(scalars, 'train_loss_step')